In [ ]:
import module1
from importlib import reload
import random
reload(module1)

ca = module1.CellularAutomata(
    img_size=6, 
    channel_count=6,
    layer_counts=[512],
    perception_kernel=module1.sobel_state_kernel())
ca.noise_range = (0, 0) #(-0.1, 0.1)
ca.conserve_mass = False
ca.edge_strategy = module1.EdgeStrategy.MIRROR

lifetime_range = (24,40)

x0 = lambda: ca.pointfilled(ca.constfilled(1.0), point_value=0.0)
xf = lambda: ca.bordered(ca.constfilled(1.0), border_value=0.0)
training = module1.init_training(ca, learning_rate=1.0e-3)
training.show_sample_run(x0, xf, 50)

while not training.is_done():
    print("Training...")
    training.run(x0, xf, random.randrange(*lifetime_range), max_seconds=5)
    training.show_sample_run(x0, xf, 50)
    training.show_loss_history()
    training.save()
    print(" -- saved parameters")